In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Simplified version: Extract differential module metrics for target species from Excel and output the top_n ranked modules.

Functionality Description:
1. Check initial conditions: Ensure Python version, necessary libraries, Excel file existence, and data integrity.
2. Read data from the Excel file and extract differential module metrics for target species.
3. Calculate differential metrics (mean and variance) for each module and perform standardization.
4. Sort modules based on combined differences and select the top_n ranked modules.
5. Save the results to an Excel file, including selected module data and combined difference rankings.
"""

import os
import sys
import importlib.util
from typing import List, Tuple, Optional
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def check_initial_conditions(file_path: str,
                             species_col: int = 0,
                             modules_start_col: int = 4,
                             target_species: Optional[List[str]] = None,
                             data_skiprows: int = 4,
                             header_skiprows: int = 3) -> bool:
    """
    Check necessary prerequisites: Python version, required libraries, Excel file existence, and data integrity.
    """
    # Check Python version
    if sys.version_info < (3, 0):
        print(f"Python 3.x or higher is required, current version: {sys.version_info.major}.{sys.version_info.minor}")
        return False

    # Check if required libraries are installed
    for pkg in ['pandas', 'numpy', 'openpyxl', 'sklearn']:
        if importlib.util.find_spec(pkg) is None:
            print(f"Missing required library: {pkg}")
            return False

    # Check if Excel file exists
    if not os.path.exists(file_path):
        print(f"Excel file does not exist: {file_path}")
        return False

    # Check data integrity
    try:
        df = pd.read_excel(file_path, na_values=["#N/A"], skiprows=data_skiprows, header=None)
        if species_col >= len(df.columns) or modules_start_col >= len(df.columns):
            print("Specified column index exceeds Excel data range.")
            return False
        if target_species:
            species_data = df.iloc[:, species_col].values
            for sp in target_species:
                if sp not in species_data:
                    print(f"Target species '{sp}' does not exist in the data.")
                    return False
    except Exception as e:
        print(f"Error reading Excel data: {e}")
        return False

    print("All initial conditions are met.")
    return True

def extract_top_modules(file_path: str,
                        species_col: int = 0,
                        modules_start_col: int = 4,
                        target_species: List[str] = None,
                        top_n: int = 40,
                        output_file: Optional[str] = None,
                        data_skiprows: int = 4,
                        header_skiprows: int = 3) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Extract differential module metrics for target species, return sorted results and selected top_n module data, and write to Excel file.
    """
    if not target_species:
        raise ValueError("Please provide at least one target species.")
        
    # Read module names row (4th row in Excel)
    try:
        header_df = pd.read_excel(file_path, na_values=["#N/A"], skiprows=header_skiprows, nrows=1, header=None)
    except Exception as e:
        raise IOError(f"Failed to read module column names: {e}")
    module_names = header_df.iloc[0, modules_start_col:].tolist()
    
    # Read actual data (starting from the 5th row)
    try:
        df_data = pd.read_excel(file_path, na_values=["#N/A"], skiprows=data_skiprows, header=None)
    except Exception as e:
        raise IOError(f"Failed to read Excel data: {e}")
    df_data.columns = list(range(df_data.shape[1]))
    
    # Rename module columns
    rename_mapping = {col: module_names[col - modules_start_col]
                      for col in range(modules_start_col, df_data.shape[1])
                      if col - modules_start_col < len(module_names)}
    df_data.rename(columns=rename_mapping, inplace=True)
    
    print("Data successfully read, number of columns:", df_data.shape[1])
    
    # Extract species names and module data
    species_names = df_data.iloc[:, species_col]
    modules_data = df_data.iloc[:, modules_start_col:]
    modules_data = modules_data.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    
    # Record the index position of target species in the DataFrame (only take the first match)
    target_indices = {}
    for sp in target_species:
        indices = species_names[species_names == sp].index
        if len(indices) == 0:
            raise ValueError(f"Target species '{sp}' not found.")
        target_indices[sp] = indices[0]
    
    module_list = []
    diff_metrics = {sp: {"mean": [], "var": []} for sp in target_species}
    
    # Calculate differential metrics for each module
    for module in modules_data.columns:
        values = modules_data[module]
        for sp in target_species:
            idx = target_indices[sp]
            target_value = values.iloc[idx]
            diffs = np.abs(values.drop(index=idx) - target_value)
            diff_metrics[sp]["mean"].append(diffs.mean())
            diff_metrics[sp]["var"].append(diffs.var())
        module_list.append(module)
    
    # Aggregate metrics: store mean and variance metrics for each module across all target species
    data_matrix = []
    for i in range(len(module_list)):
        row = []
        for sp in target_species:
            row.append(diff_metrics[sp]["mean"][i])
            row.append(diff_metrics[sp]["var"][i])
        data_matrix.append(row)
    data_matrix = np.array(data_matrix)
    
    # Standardize and calculate combined differences
    scaler = StandardScaler()
    scaled = scaler.fit_transform(data_matrix)
    combined = scaled.sum(axis=1)
    
    # Create result DataFrame and sort
    result_df = pd.DataFrame({
        'Module': module_list,
        'Combined Difference': combined
    }).sort_values(by='Combined Difference', ascending=False)
    
    # Select top_n modules
    top_modules = result_df.head(top_n)['Module'].tolist()
    print(f"Selected top {top_n} modules:", top_modules)
    
    try:
        selected = [df_data.columns[species_col]] + top_modules
        top_data = df_data[selected]
    except KeyError as e:
        raise KeyError(f"Specified column name error: {e}")
    
    # Save results to Excel file
    if output_file is None:
        desktop = os.path.join(os.path.expanduser("~"), "Desktop")
        output_file = os.path.join(desktop, f'top_{top_n}_modules_data.xlsx')
    
    with pd.ExcelWriter(output_file) as writer:
        top_data.to_excel(writer, sheet_name='Top Modules Data', index=False)
        result_df.to_excel(writer, sheet_name='Combined Differences', index=False)
    
    print("Data saved to:", output_file)
    return result_df, top_data

# Main entry point for testing and actual execution
if __name__ == "__main__":
    # Modify the following parameters as needed
    file_path = r'C:\Users\lichi\Desktop\工作簿1.xlsx'
    target_species = ['Aestuariirhabdus_salina_LZHN29T', 'Thalassotalea_salina_PLHSN55T']
    species_col = 0              # Column where species names are located
    modules_start_col = 4        # Starting column for module data (starting from the 5th column in Excel)
    top_n = 40                   # Select top 40 modules
    data_skiprows = 4            # Data starts from the 5th row
    header_skiprows = 3          # Module names are located in the 4th row

    if check_initial_conditions(file_path, species_col, modules_start_col,
                                target_species, data_skiprows, header_skiprows):
        print("Condition check passed, starting data analysis...")
        result_df, top_data = extract_top_modules(file_path,
                                                   species_col,
                                                   modules_start_col,
                                                   target_species,
                                                   top_n,
                                                   data_skiprows=data_skiprows,
                                                   header_skiprows=header_skiprows)


All initial conditions are met.
Condition check passed, starting data analysis...
Data successfully read, number of columns: 301
Selected top 40 modules: ['M00555 Betaine biosynthesis, choline => betaine', 'M00554 UDP-Gal biosynthesis, Gal => UDP-Gal', 'M00144 NADH:quinone oxidoreductase, prokaryotes', 'M00948 Hydroxyproline degradation, trans-4-hydroxy-L-proline => 2-oxoglutarate', 'M00854 Glycogen biosynthesis, glucose-1P => glycogen/starch', 'M00038 Tryptophan metabolism, tryptophan => kynurenine => 2-aminomuconate', 'M00156 Cytochrome c oxidase, cbb3-type', 'M00565 Trehalose biosynthesis, D-glucose 1P => trehalose', 'M00529 Denitrification, nitrate => nitrogen', 'M00122 Cobalamin biosynthesis, cobyrinate a,c-diamide => cobalamin', 'M01000 GDP-Man biosynthesis, Fru-6P => GDP-Man', 'M00632 Galactose degradation, Leloir pathway, galactose => alpha-D-glucose-1P', 'M00793 dTDP-L-rhamnose biosynthesis, glucose-1P => dTDP-L-Rha', 'M00855 Glycogen degradation, glycogen => glucose-6P', 'M00